<a href="https://colab.research.google.com/github/RT-UNAB/Programacion/blob/main/CNN_SkinTears_v4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# VERIFICO SI ESTA ACTIVA LA GPU

import tensorflow as tf
print ("La GPU esta activa???     ", tf.test.is_gpu_available(),"\n\n")


# CONSTRUYENDO LA CNN (Convolutional Neural Network)
# IMPORTO LAS LIBRERIAS

# https://medium.com/nybles/create-your-first-image-recognition-classifier-using-cnn-keras-and-tensorflow-backend-6eaab98d14dd

# IMPORTO LIBRERIAS Y PAQUETES DE KERAS

from keras.models import Sequential
from keras.layers import Convolution2D, MaxPooling2D, Flatten, Dense, Dropout

# MONTO MI GOOGLE DRIVE, PARA POSTERIORMENTE LEER LAS IMAGENES
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

La GPU esta activa???      True 




Using TensorFlow backend.


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [ ]:
# PREPROCESO LAS IMAGENES GENERANDO MAS DATA

from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1/255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True,
                                   vertical_flip = True,
                                   validation_split=0.1)

test_datagen = ImageDataGenerator(rescale = 1/255)



training_set = train_datagen.flow_from_directory(
    directory='/content/gdrive/My Drive/SkinTears/dataset/training_set', 
    target_size = (128, 128), 
    batch_size = 32, 
    class_mode = 'categorical')

test_set = test_datagen.flow_from_directory(
    directory='/content/gdrive/My Drive/SkinTears/dataset/test_set', 
    target_size = (128, 128), 
    batch_size = 32, 
    class_mode = 'categorical')


Found 16444 images belonging to 4 classes.
Found 4166 images belonging to 4 classes.


In [ ]:

# Initialising the CNN
classifier = Sequential()

# PASO 1 - CONVOLUTION
# extrae características de la imagen de entrada. preserva la relación espacial entre los píxeles al aprender las características de la imagen utilizando pequeños cuadrados de datos de entrada.
classifier.add(Convolution2D(32, 3, 3, input_shape = (128, 128, 3), activation = 'relu'))
classifier.add(Convolution2D(32, 3, 3, activation = 'relu'))
classifier.add(Convolution2D(32, 3, 3, activation = 'relu'))

# PASO 2 - POOLING
# también llamada submuestreo o submuestreo, reduce la dimensionalidad de cada mapa de características, pero conserva la información más importante.
# En el caso de Max Pooling, definimos una vecindad espacial (por ejemplo, una ventana de 2 × 2) y tomamos el elemento más grande del mapa de características rectificado dentro de esa ventana
classifier.add(MaxPooling2D(pool_size = (2, 2)))

# SE AGREGA UNA SEGUNDA CAPA CONVOLUTIONAL
classifier.add(Convolution2D(64, 3, 3, activation = 'relu'))
classifier.add(Convolution2D(64, 3, 3, activation = 'relu'))
classifier.add(MaxPooling2D(pool_size = (2, 2)))

# SE AGREGA UNA TERCERA CAPA CONVOLUTIONAL
classifier.add(Convolution2D(128, 3, 3, activation = 'relu'))
classifier.add(Convolution2D(128, 3, 3, activation = 'relu'))
classifier.add(MaxPooling2D(pool_size = (2, 2)))




#*************************************************************

# SE AGREGA UNA CUARTA CAPA CONVOLUTIONAL
classifier.add(Convolution2D(256, 3, 3, activation = 'relu'))
classifier.add(Convolution2D(256, 3, 3, activation = 'relu'))
classifier.add(MaxPooling2D(pool_size = (2, 2)))


#*************************************************************







# PASO 3 - FLATTENING
# Aquí la matriz se convierte en una matriz lineal de modo que se ingrese en los nodos de nuestra red neuronal.
classifier.add(Flatten())

# PASO 4 - FULL CONNECTION
# conectamos nuestra red convolucional a una red neuronal para luego compilar nuestra red.



# hemos creado una red neuronal de 2 capas con una función softmax como función de activación para la última capa.
# ya que necesitamos encontrar la probabilidad de que la imagens sea T0, T1, T2 o T3.
classifier.add(Dropout(0.3))
classifier.add(Dense(output_dim = 1024, activation = 'relu'))
classifier.add(Dropout(0.3))
classifier.add(Dense(output_dim = 4, activation="softmax"))



# COMPILO LA CNN
classifier.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

classifier.fit_generator(
    training_set, 
    nb_epoch = 30)



# GUARDO EL CLASIFICADOR EN UN ARCHIVO H5
# para analizar imagenes rapidamente invocando este modelo.
classifier.save('/content/gdrive/My Drive/SkinTears/modelo_SkinTears_v4.h5')



W0904 16:56:51.531615 139883980339072 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:66: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), input_shape=(128, 128,..., activation="relu")`
  """
W0904 16:56:51.546093 139883980339072 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:541: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0904 16:56:51.556627 139883980339072 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4432: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: UserWarning: Update your

Epoch 1/30
514/514 [==============================] - 9822s 19s/step - loss: 0.9713 - acc: 0.5455
Epoch 2/30
514/514 [==============================] - 511s 994ms/step - loss: 0.6251 - acc: 0.7439
Epoch 3/30
514/514 [==============================] - 498s 969ms/step - loss: 0.5520 - acc: 0.7758
Epoch 4/30
514/514 [==============================] - 499s 971ms/step - loss: 0.4075 - acc: 0.8408
Epoch 5/30
514/514 [==============================] - 491s 954ms/step - loss: 0.3315 - acc: 0.8734
Epoch 6/30
514/514 [==============================] - 493s 960ms/step - loss: 0.2677 - acc: 0.9002
Epoch 7/30
514/514 [==============================] - 492s 957ms/step - loss: 0.2222 - acc: 0.9186
Epoch 8/30
514/514 [==============================] - 506s 984ms/step - loss: 0.2330 - acc: 0.9131
Epoch 9/30
514/514 [==============================] - 505s 983ms/step - loss: 0.1948 - acc: 0.9279
Epoch 10/30
514/514 [==============================] - 511s 995ms/step - loss: 0.1788 - acc: 0.9382
Epoch 11/3

In [ ]:
# Fitting the CNN to the images

from tensorflow import keras
classifier = keras.models.load_model('/content/gdrive/My Drive/SkinTears/modelo_SkinTears_v4.h5')

# Evaluacion del conjunto de imagenes de test
classifier.evaluate_generator(test_set, verbose=1)

131/131 [==============================] - 2377s 18s/step - loss: 0.8735 - acc: 0.7345


[0.8734798305134737, 0.7345175]

In [ ]:
# VERIFICO QUE PUEDO GUARDAR Y LEER UN ARCHIVO ALMACENADO EN MI GOOGLE DRIVE COMO VERIFICACION DE CONECCION

with open('/content/gdrive/My Drive/CNNExamples/Ej4/dataset/LINKS.txt', 'w') as f:
  f.write('Hola Google Drive!')
!cat /content/gdrive/My\ Drive/CNNExamples/Ej4/dataset/LINKS.txt

Hello Google Drive!

In [ ]:
"""
# HAGO PREDICCIONES CON EL MODELO GUARDADO EXTENSION H5 POR EL CLASIFICADOR SOBRE UNA IMAGEN PUNTUAL
from tensorflow import keras

classifier = keras.models.load_model('/content/gdrive/My Drive/Colab Notebooks/clasificador_entrenado_SkinTears10000_4Class.h5')


import numpy as np
from keras.preprocessing import image
test_image = image.load_img('/content/gdrive/My Drive/CNNExamples/SkinTears/dataset/test_set/T2/T2.40.jpg', target_size = (64, 64))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = classifier.predict(test_image)
training_set.class_indices
if result[0][0] >= 0.5:
  prediction = 'dog'
else:
  prediction = 'cat'
print(prediction)
"""


In [ ]:
# HAGO PREDICCIONES CON EL MODELO GUARDADO DE EXTENSION H5, SOBRE UNA IMAGEN PUNTUAL
from tensorflow import keras

classifier = keras.models.load_model('/content/gdrive/My Drive/SkinTears/modelo_SkinTears.h5')


import numpy as np
from keras.preprocessing import image
test_image = image.load_img('/content/gdrive/My Drive/SkinTears/dataset/test_set/Tipo0/SkinTear_T0_20.jpg', target_size = (64, 64))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = classifier.predict(test_image)

#training_set.class_indices

clase = np.argmax(result, axis=-1)
conf = np.max(result, axis=-1)
print (result, conf)





[[1. 0. 0. 0.]] [1.]


In [ ]:
# CONVIERTO MODELO H5 A TFLITE METODO 1 ----------- FUNCIONA OK ! ! !

"""

import tensorflow as tf
from tensorflow import lite

converter = lite.TFLiteConverter.from_keras_model_file("/content/gdrive/My Drive/Colab Notebooks/clasificador_entrenado_SkinTears10000_4ClassDM.h5")
tflite_model = converter.convert()
open("/content/gdrive/My Drive/Colab Notebooks/converted_model.tflite", "wb").write(tflite_model)


"""

In [ ]:
# CONVIERTO MODELO H5 A PB METODO 1 ----------- FUNCIONA OK ! ! !
# This was created with @warptime's help. Thank you!

from tensorflow.python.framework import graph_util
from tensorflow.python.framework import graph_io
from keras.models import load_model
from keras import backend as K
import os.path as osp
import os
import tensorflow as tf


model = load_model('/content/gdrive/My Drive/SkinTears/modelo_SkinTears.h5')
nb_classes = 4 # The number of output nodes in the model
prefix_output_node_names_of_final_network = 'output_node_NEW'

K.set_learning_phase(0)

pred = [None]*nb_classes
pred_node_names = [None]*nb_classes
for i in range(nb_classes):
    pred_node_names[i] = prefix_output_node_names_of_final_network+str(i)
    pred[i] = tf.identity(model.output[i], name=pred_node_names[i])
print('output nodes names are: ', pred_node_names)

sess = K.get_session()
output_fld = 'tensorflow_model/'
if not os.path.isdir(output_fld):
    os.mkdir(output_fld)
output_graph_name = '/content/gdrive/My Drive/SkinTears/saved_model_path' + '.pb'
output_graph_suffix = '_inference'

constant_graph = graph_util.convert_variables_to_constants(sess, sess.graph.as_graph_def(), pred_node_names)
graph_io.write_graph(constant_graph, output_fld, output_graph_name, as_text=False)
print('saved the constant graph (ready for inference) at: ', osp.join(output_fld, output_graph_name))


W0816 18:31:51.936748 140637674309504 deprecation.py:323] From <ipython-input-4-d0a9d8bc6f95>:31: convert_variables_to_constants (from tensorflow.python.framework.graph_util_impl) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.compat.v1.graph_util.convert_variables_to_constants`
W0816 18:31:51.937840 140637674309504 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/graph_util_impl.py:270: extract_sub_graph (from tensorflow.python.framework.graph_util_impl) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.compat.v1.graph_util.extract_sub_graph`


output nodes names are:  ['output_node_NEW0', 'output_node_NEW1', 'output_node_NEW2', 'output_node_NEW3']
saved the constant graph (ready for inference) at:  /content/gdrive/My Drive/Colab Notebooks/NEW_TEST_saved_model_path.pb


In [ ]:
# PROBAR QUE HACE ESTE CODIGO


import tensorflow as tf
path = "ruta/al/modelo.pb"
#Cargamos el archivo pb como graphdef
with tf.io.gfile.GFile(path, "rb") as fid:
  graph_def = tf.GraphDef()
  graph_def.ParseFromString(fid.read())

graph = tf.get_default_graph()
tf.import_graph_def(graph_def, name="heridas")

entrada = graph.get_tensor_by_name("heridas/conv2d_1_input:0")
salida = graph.get_tensor_by_name("heridas/nombre_out:0")

with tf.Session(graph=graph) as sess:
  resultado = sess.run(salida, feed_dict={entrada:imagen})
  print(resultado) #[1,0,0,0]

In [ ]:
import tensorflow as tf
print(tf.__version__)


1.14.0


In [ ]:
# INDICAR LA RUTA DEL ARCHIVO PB

filename = "/content/gdrive/My Drive/SkinTears/modelo_SkinTears.pb"
import tensorflow as tf
g = tf.GraphDef()
g.ParseFromString(open(filename, 'rb').read())
print()
print(filename)
print("=======================INPUT=========================")
print([n for n in g.node if n.name.find('input') != -1])
print("=======================OUTPUT========================")
print([n for n in g.node if n.name.find('output') != -1])
print("===================KERAS_LEARNING=====================")
print([n for n in g.node if n.name.find('keras_learning_phase') != -1])
print("======================================================")
print()



/content/gdrive/My Drive/Colab Notebooks/TEST_saved_model_path.pb
=======================INPUT=========================
[name: "conv2d_1_input"
op: "Placeholder"
attr {
  key: "dtype"
  value {
    type: DT_FLOAT
  }
}
attr {
  key: "shape"
  value {
    shape {
      dim {
        size: -1
      }
      dim {
        size: 128
      }
      dim {
        size: 128
      }
      dim {
        size: 3
      }
    }
  }
}
, name: "dropout_1/keras_learning_phase/input"
op: "Const"
attr {
  key: "dtype"
  value {
    type: DT_BOOL
  }
}
attr {
  key: "value"
  value {
    tensor {
      dtype: DT_BOOL
      tensor_shape {
      }
      bool_val: false
    }
  }
}
]
=======================OUTPUT========================
[name: "output_node_NEW0"
op: "Identity"
input: "strided_slice"
attr {
  key: "T"
  value {
    type: DT_FLOAT
  }
}
, name: "output_node_NEW1"
op: "Identity"
input: "strided_slice_1"
attr {
  key: "T"
  value {
    type: DT_FLOAT
  }
}
, name: "output_node_NEW2"
op: "Identi

In [ ]:
import os

import tensorflow as tf
from tensorflow import keras

saved_model_path="/content/gdrive/My Drive/SkinTears/TEST_saved_model_path.pb"


"""
new_model = tf.contrib.saved_model.load_keras_model(saved_model_path)
"""


new_model = tf.keras.experimental.load_from_saved_model(saved_model_path)


FailedPreconditionError: ignored